In [ ]:
!pip install groq
!pip install google-api-python-client
!pip install pandas

In [2]:
import pandas as pd
import googleapiclient.discovery
from groq import Groq

In [3]:
api_service_name = 'youtube'
api_version = 'v3'
YT_API_KEY = 'your yt api key'
GROQ_API_KEY = "your groq key"

In [4]:
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=YT_API_KEY)
groqclient = Groq(api_key=GROQ_API_KEY)

In [6]:

allcomments = []

def getAllComments():

  nextPageToken = ""

  while(True):
    request = youtube.commentThreads().list(
        part=['snippet', 'replies'],
        videoId='fsvN4IiZFCA',
        maxResults='100',
        pageToken=nextPageToken)

    response = request.execute()
    allcomments.extend(response["items"])

    if('nextPageToken' not in response.keys()):
      break
    else:
      nextPageToken = response["nextPageToken"]

  return allcomments

allcomments = getAllComments()

In [7]:
# print(response["items"][1].keys())

In [8]:
commentlist = []
comments = ""
counter = 1

for item in allcomments:
    comment = item["snippet"]["topLevelComment"]["snippet"]
    comments += "{}. {}\n".format(counter, comment['textDisplay'])
    counter+=1

    commentlist.append([comment["authorDisplayName"],
                       comment["publishedAt"],
                       comment["updatedAt"],
                       comment["likeCount"],
                       comment["textDisplay"]
                   ])

    """ code to handle comment replies below as comment """
    if(item["snippet"]["totalReplyCount"]):
      for commentreply in item["replies"]["comments"]:
        comments += "{}. {}\n".format(counter, commentreply["snippet"]["textDisplay"])
        counter += 1
        commentlist.append([commentreply["snippet"]["authorDisplayName"],
                            commentreply["snippet"]["publishedAt"],
                            commentreply["snippet"]["updatedAt"],
                            commentreply["snippet"]["likeCount"],
                            commentreply["snippet"]["textDisplay"]])

df = pd.DataFrame(commentlist, columns=["author", "published_at", "updated_at", "like_count", "text"])

In [9]:
len(df)

124

In [10]:
user_question = "Give me a summary of the comments."

In [11]:
prompt = """
          Based on the comments, answer the question.
          {user_question}

          Comments:
          {comments}
""".format(user_question=user_question, comments=comments)

In [12]:
completion = groqclient.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant who helps the user to answer the question from the context information given."
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)



In [13]:
# This code is for stream output
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Based on the comments, here is a summary:

* The majority of commenters loved the episode and thought Tessa was amazing, funny, and adorable. They appreciated her energy, humor, and storytelling.
* Many commenters requested for Tessa to come back on the podcast again, and some even suggested making her a regular member.
* A few commenters mentioned that they could relate to Tessa's experiences, especially when it came to stalking and university life in the UK.
* Some commenters made fun of or teased other podcast members, like Fahim, and joked about his absence or potential conflicts with Sharan.
* A few commenters mentioned specific moments or quotes from the episode that they enjoyed, such as Tessa's Roommate story or her opinions on Harry Potter.
* There were also some random comments and jokes that didn't seem to be related to the episode or podcast.

Overall, the comments suggest that the episode was well-received and that Tessa was a hit with the audience.